In [1]:
import openai
import pandas as pd
import tqdm
import glob

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open("../apikey/apikey.txt", "r") as f:
    openai.api_key = f.readline().replace("\n", "")

In [3]:
def query_prompt(prompt, max_tokens=1000):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=0,
    )
    return response["choices"][0]["message"]["content"]

In [4]:
import random

In [5]:
def get_df(fname):
    def f(categories):
        for cat in categories:
            for word in [
                "geology",
                "physics",
                "chemistry",
                "mathematical",
                "biology",
                "astronomy",
                "ecology",
                "genetics",
                "statistics",
                "theoretical"
            ]:
                if word.lower() in cat.lower():
                    return True
            
        return False
    
    def text_preprocess(text):
        return text.replace("====", "\n\n").replace("===", "\n\n").replace("==", "\n\n")

    def sep_n(text, n=40):
        try:
            text = text.split(".")
            start_index = random.randint(0, len(text) - n)
            return ".".join(text[start_index:start_index+n])
        except:
            return None
    
    df = pd.read_parquet(fname)
    df_science = df[df["category"].apply(f)]
    df_science["text"] = df_science["text"].apply(text_preprocess)
    df_science["text"] = df_science["text"].apply(sep_n)
    df_science = df_science[df_science["text"].notnull()]
    return df_science.sample(len(df_science)//5)

In [6]:
files = glob.glob("../data/wikipedia_fixed/*.parquet")

In [7]:
import time

In [8]:
import pickle

In [9]:
import json

In [10]:
from datetime import datetime as dt
import os

In [11]:
texts = []

In [25]:
import traceback 
batch_size = 1

def make_prompt(series):
    prompt = f"""
You are a professor at a science university and are creating a test for your students.
Please create one exam based on the provided text, where you choose the most accurately statement from five options for the question.
The output should json format below:
{{"prompt": <the question text>, "answer" <answer(one of A through E)>, "A": <option A>, "B": <option B>, "C": <option C>, "D": <option D>, "E": <option E>}}

Context:
{series['text']}

Attention:
- The five opinions should be LONG sentences.

"""
    return prompt

def f(series):
    try:
        if series["A"] != series["A"]:
            if type(series["answer"]) == dict:
                for key in ["A", "B", "C", "D", "E"]:
                    series[key] = series["choices"][key]
            elif type(series["answer"] == list):
                for i, key in enumerate(["A", "B", "C", "D", "E"]):
                    series[key] = series["choices"][i]
    except:
        return series
    return series

now_date = dt.now().strftime("%Y%m%d%H%M%S")

first = True
for file in files:
    if os.path.basename(file) in ["all.parquet"]:
        print(f"pass: {file}")
        continue
    df_science = get_df(file)
    
    for i in tqdm.tqdm(range(len(df_science)), desc=file):
        try:
            series = df_science.iloc[i]
            prompt = make_prompt(series)
            text = query_prompt(prompt)
            texts_json = json.loads(text)
            if first:
                print(texts_json)
                first = False
            if type(texts_json) == dict:
                texts_json["wiki_id"] = series["id"]
                texts_json["original_text"] = series["text"]
                texts.append(texts_json)
                for col in ["A", "B", "C", "D", "E", "answer", "prompt"]:
                    if col not in texts_json:
                        print(f"{col} not existed: {texts_json}")
            else:
                for text_json in texts_json:
                    text_json["wiki_id"] = series["id"]
                    text_json["original_text"] = series["text"]
                    texts.append(text_json)
        except Exception as e:
            print(e)
            traceback.print_exc()
            print(text)
            time.sleep(10)
        if i % 20 == 0:
            df_texts = pd.DataFrame(texts)
            df_texts = df_texts.apply(f, axis=1)

            df_texts.to_parquet(f"output_gpt3.5_generate/{now_date}.parquet")

../data/wikipedia_fixed/a.parquet:   0%|▏                                                                                 | 1/388 [00:05<35:05,  5.44s/it]

{'prompt': "What is the title of Alan Tutton Johns' PhD thesis?", 'answer': 'C', 'A': 'The mechanism of propionic acid formation in fermentation', 'B': 'Advanced study in U.S.', 'C': 'The mechanism of propionic acid formation in fermentation with special reference to the rumen of the sheep', 'D': 'Honorary doctorate citation, Alan Johns, 1985', 'E': 'No. 47237'}


../data/wikipedia_fixed/a.parquet:   4%|███▌                                                                             | 17/388 [02:34<54:38,  8.84s/it]

Invalid \escape: line 1 column 72 (char 71)
{"prompt": "Which of the following statements is true about the Σ 1 0 {\displaystyle \Sigma _{1}^{0}} sets?", 
"answer": "C", 
"A": "Σ 1 0 {\displaystyle \Sigma _{1}^{0}} sets are closed sets in the usual topology on Baire space or Cantor space.", 
"B": "Σ 1 0 {\displaystyle \Sigma _{1}^{0}} sets are effectively open sets in the usual topology on Baire space or Cantor space.", 
"C": "Σ 1 0 {\displaystyle \Sigma _{1}^{0}} sets can be defined in the language of Peano arithmetic by a Σ 1 0 {\displaystyle \Sigma _{1}^{0}} formula.", 
"D": "Σ 1 0 {\displaystyle \Sigma _{1}^{0}} sets are subsets of Baire space or Cantor space that are computably enumerable.", 
"E": "Σ 1 0 {\displaystyle \Sigma _{1}^{0}} sets are sometimes called effectively closed sets."}


Traceback (most recent call last):
  File "/tmp/ipykernel_15213/3124827155.py", line 47, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Invalid \escape: line 1 column 72 (char 71)
../data/wikipedia_fixed/a.parquet:  10%|████████▎                                                                        | 40/388 [05:59<50:43,  8.75s/it]

...

../data/wikipedia_fixed/w.parquet:  12%|█████████▎                                                                       | 14/121 [01:30<13:23,  7.51s/it]

Invalid \escape: line 1 column 159 (char 158)
{"prompt": "What is the basis of Tajima's D?", "answer": "C", "A": "Watterson's estimator is commonly used for its simplicity.", "B": "The estimate of θ ^ w {\displaystyle {\widehat {\theta \,}}_{w}} , often denoted as θ ^ w {\displaystyle {\widehat {\theta \,}}_{w}} , is θ ^ w = K a n , {\displaystyle {\widehat {\theta \,}}_{w}={K \over a_{n}},} where K {\displaystyle K} is the number of segregating sites (an example of a segregating site would be a single-nucleotide polymorphism) in the sample and a n = ∑ i = 1 n − 1 1 i {\displaystyle a_{n}=\sum _{i=1}^{n-1}{1 \over i}} is the ( n − 1 ) {\displaystyle (n-1)} th harmonic number.", "C": "Comparing the value of the Watterson's estimator, to nucleotide diversity is the basis of Tajima's D which allows inference of the evolutionary regime of a given locus.", "D": "The assumptions made are that there is a sample of n {\displaystyle n} haploid individuals from the population of interest, that t

Traceback (most recent call last):
  File "/tmp/ipykernel_15213/3124827155.py", line 47, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Invalid \escape: line 1 column 159 (char 158)
../data/wikipedia_fixed/w.parquet:  74%|████████████████████████████████████████████████████████████▏                    | 90/121 [09:29<03:14,  6.27s/it]Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 755, in _interpret_response_line
    data = json.loads(rbody)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)

HTTP code 502 from API (<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>
)
{"prompt": "Which of the following statements is the most accurate based on the provided text?", "answer": "C", "A": "The Pittsburgh Post-Gazette published an article titled 'Prince of Puzzles' on October 25, 1999.", "B": "The Philadelphia Inquirer reported on the 2009 Inquirer Sudoku National Championship.", "C": "The MIT Mystery Hunt 2019 Wrapup video can be found on YouTube.", "D": "BoardGameGeek features information about the game 'Roll for the Galaxy'.", "E": "The Washington Post published an article on March 9, 1993, about the top 10 finalists in Westinghouse's competition."}


../data/wikipedia_fixed/z.parquet: 100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [02:32<00:00,  6.62s/it]


In [27]:
df_texts = pd.DataFrame(texts)
df_texts = df_texts.apply(f, axis=1)

df_texts.to_parquet(f"output_gpt3.5_generate/{now_date}.parquet")

In [28]:
len(df_texts)

5839